# Predict low high models
Model selection for predict lo hi


To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
#%pip install binance-connector==1.13.0
%pip install pandas
%pip install scikit-learn
%pip install tensorflow==2.11.0
%pip install keras==2.11.0
%pip install scikeras


import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Feature engineering
Read data from bot's history, resample to equal intervals, create X, y

In [2]:
import os
import glob
import pandas as pd
from datetime import timedelta,date

def read_last_data(strategy):
    """ Read last last day from bot history """
    
    def read_last(symbol: str, name: str):
        data_dir=f"./../data/{strategy}/Xy"
        file_path = sorted([f for f in os.listdir(data_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
        return pd.read_csv(f"{data_dir}/{file_path}", parse_dates=True, index_col="datetime")

    data=read_last("BTCUSDT", "data")
    X=read_last("BTCUSDT", "X")
    y=read_last("BTCUSDT", "y")
    return data,X,y

#strategy="SimpleKerasStrategy"
strategy="LSTMStrategy"
bidask,X_bot,y_bot = read_last_data(strategy)
bidask

,symbol,bid,bid_vol,ask,ask_vol
datetime,,,,,
2023-05-05 02:33:12.632865,BTCUSDT,29231.03,0.04615,29231.04,2.78041
2023-05-05 02:33:13.284107,BTCUSDT,29231.03,0.16113,29231.04,2.78041
2023-05-05 02:33:13.375662,BTCUSDT,29231.03,0.16113,29231.04,2.60088
2023-05-05 02:33:13.467185,BTCUSDT,29231.03,0.16113,29231.04,2.37988
2023-05-05 02:33:13.557162,BTCUSDT,29231.03,0.15903,29231.04,2.37988
...,...,...,...,...,...
2023-05-05 02:38:12.917288,BTCUSDT,29242.64,1.55770,29242.65,8.70239
2023-05-05 02:38:13.023607,BTCUSDT,29242.64,1.59787,29242.65,8.70239
2023-05-05 02:38:13.230509,BTCUSDT,29242.64,1.55770,29242.65,8.70239


In [3]:

def get_targets(bidask):
        """ 
        Calculate targets - bid/ask bounds in future prediction window
        """
        predict_window="60s"
        fut_min=bidask[["bid", "ask"]][::-1].rolling(predict_window).min()[::-1].rename(columns={"bid":"bid_min_fut", "ask":"ask_min_fut"})
        fut_max=bidask[["bid", "ask"]][::-1].rolling(predict_window).max()[::-1].rename(columns={"bid":"bid_max_fut", "ask":"ask_max_fut"})
        return pd.concat([fut_min, fut_max], axis = 1)

y_raw = get_targets(bidask)
y_raw


,bid_min_fut,ask_min_fut,bid_max_fut,ask_max_fut
datetime,,,,
2023-05-05 02:33:12.632865,29231.03,29231.04,29242.77,29242.78
2023-05-05 02:33:13.284107,29231.03,29231.04,29242.77,29242.78
2023-05-05 02:33:13.375662,29231.03,29231.04,29242.77,29242.78
2023-05-05 02:33:13.467185,29231.03,29231.04,29242.77,29242.78
2023-05-05 02:33:13.557162,29231.03,29231.04,29242.77,29242.78
...,...,...,...,...
2023-05-05 02:38:12.917288,29242.64,29242.65,29242.64,29242.65
2023-05-05 02:38:13.023607,29242.64,29242.65,29242.64,29242.65
2023-05-05 02:38:13.230509,29242.64,29242.65,29242.64,29242.65


Resample data to use equal time intervals

In [4]:
def resampled(X: pd.DataFrame, y: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """ Resample to make equal intervals time series """
    time_interval="10s"
    Xy=pd.concat([X,y], axis=1)
    diffmap = [(c, "sum") for c in Xy.columns if c.endswith("diff")]
    timemap = [(c, "last") for c in Xy.columns if c.startswith("time") and not c.endswith("diff")]
    l2map = [(c, "last") for c in Xy.columns if c.startswith("l2_")]
    
    futmap = [(c, "last") for c in Xy.columns if c.endswith("_fut")]
    colmap = dict(diffmap + timemap + l2map + futmap + [("spread", "last")])
    resampled = Xy.resample(time_interval).agg(colmap).dropna()
    return resampled[X.columns], resampled[y.columns]
    
#X,y = resampled(X_bot, y_raw)
X,y=X_bot,y_raw
pd.concat([X,y], axis=1).dropna()

,time_hour,time_minute,time_second,time_day_of_week,time_diff,spread,bid_diff,bid_vol_diff,ask_diff,ask_vol_diff,l2_bucket_-10,l2_bucket_-9,l2_bucket_-8,l2_bucket_-7,l2_bucket_-6,l2_bucket_-5,l2_bucket_-4,l2_bucket_-3,l2_bucket_-2,l2_bucket_-1,l2_bucket_0,l2_bucket_1,l2_bucket_2,l2_bucket_3,l2_bucket_4,l2_bucket_5,l2_bucket_6,l2_bucket_7,l2_bucket_8,l2_bucket_9,bid_min_fut,ask_min_fut,bid_max_fut,ask_max_fut
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-05 02:33:12.632865,2,33,12,4,430186000.0,0.01,0.0,0.00000,0.0,-0.01432,0.00082,0,0.0,0,0,0.0,0.0,0.0,0.0,2.68462,3.78257,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,29231.03,29231.04,29242.77,29242.78
2023-05-05 02:33:13.284107,2,33,13,4,651242000.0,0.01,0.0,0.11498,0.0,0.00000,0.00082,0,0.0,0,0,0.0,0.0,0.0,0.0,2.68462,3.78257,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,29231.03,29231.04,29242.77,29242.78
2023-05-05 02:33:13.375662,2,33,13,4,91555000.0,0.01,0.0,0.00000,0.0,-0.17953,0.00082,0,0.0,0,0,0.0,0.0,0.0,0.0,2.68462,3.78257,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,29231.03,29231.04,29242.77,29242.78
2023-05-05 02:33:13.467185,2,33,13,4,91523000.0,0.01,0.0,0.00000,0.0,-0.22100,0.00082,0,0.0,0,0,0.0,0.0,0.0,0.0,2.68462,3.78257,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,29231.03,29231.04,29242.77,29242.78
2023-05-05 02:33:13.557162,2,33,13,4,89977000.0,0.01,0.0,-0.00210,0.0,0.00000,0.00082,0,0.0,0,0,0.0,0.0,0.0,0.0,2.68462,3.78257,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,29231.03,29231.04,29242.77,29242.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-05 02:38:12.917288,2,38,12,4,104831000.0,0.01,0.0,0.00000,0.0,-0.13974,0.00082,0,0.0,0,0,0.0,0.0,0.0,0.0,95.09356,1.37398,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,29242.64,29242.65,29242.64,29242.65
2023-05-05 02:38:13.023607,2,38,13,4,106319000.0,0.01,0.0,0.04017,0.0,0.00000,0.00082,0,0.0,0,0,0.0,0.0,0.0,0.0,95.09356,1.37398,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,29242.64,29242.65,29242.64,29242.65
2023-05-05 02:38:13.230509,2,38,13,4,206902000.0,0.01,0.0,-0.04017,0.0,0.00000,0.00082,0,0.0,0,0,0.0,0.0,0.0,0.0,95.09356,1.37398,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,29242.64,29242.65,29242.64,29242.65


## Create the model


In [5]:
from keras import Sequential, Input
from keras.layers import Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from scikeras.wrappers import KerasRegressor
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.compose import ColumnTransformer, make_column_selector


test_size=0.3
test_index = int(len(X)*(1-test_size))
X_train, y_train, X_test, y_test = X.iloc[:test_index], y.iloc[:test_index], X.iloc[test_index:], y.iloc[test_index:]

   
def create_model(X_train, y_train, window_size):
        
        model = Sequential()
        model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_train.shape[1])))
        model.add(Dropout(0.2))
        model.add(LSTM(32))         
        model.add(Dropout(0.2))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(y_train.shape[1], activation='linear'))
        #model.add(Dense(y_train.shape[1], activation='softmax'))
        model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])
        return model
    
    
x_scaler, y_scaler = StandardScaler().fit(X_train), StandardScaler().fit(y_train)
X_train = x_scaler.transform(X_train)
y_train = y_scaler.transform(y_train)

window_size = 10
gen = TimeseriesGenerator(X_train, y_train, length=window_size,
                               sampling_rate=1, batch_size=1)

model = create_model(X_train, y_train, window_size)
model.fit(gen,steps_per_epoch=1000, epochs=10)

2023-05-05 05:39:01.885694: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 05:39:02.017098: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-05 05:39:02.017121: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-05 05:39:02.605076: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

Epoch 1/10
1000/1000 [==============================] - 10s 7ms/step - loss: 0.2822 - mean_squared_error: 0.1926
Epoch 2/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.2047 - mean_squared_error: 0.0957
Epoch 3/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1823 - mean_squared_error: 0.0794
Epoch 4/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1558 - mean_squared_error: 0.0603
Epoch 5/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1399 - mean_squared_error: 0.0522
Epoch 6/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1309 - mean_squared_error: 0.0477
Epoch 7/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1212 - mean_squared_error: 0.0451
Epoch 8/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1291 - mean_squared_error: 0.0566
Epoch 9/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1118 - mean_squared_error